Model 1: Decision Tree

In [ ]:
# ============================================
# IMPORTS
# ============================================

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score, KFold
from collections import Counter

In [ ]:
# ============================================
# FEATURE EXTRACTION
# ============================================

def extract_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    features = []
    
    # Color histograms (8 bins per channel)
    for i in range(3):
        hist = cv2.calcHist([img_rgb], [i], None, [8], [0, 256])
        hist = hist.flatten() / hist.sum()
        features.extend(hist)
    
    # Edge density
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges > 0) / edges.size
    features.append(edge_density)
    
    # Color statistics
    for i in range(3):
        features.append(np.mean(img_rgb[:,:,i]))
        features.append(np.std(img_rgb[:,:,i]))
    
    # Brightness and contrast
    features.append(np.mean(gray))
    features.append(np.std(gray))
    
    return np.array(features)

In [ ]:
# ============================================
# LOAD DATASET
# ============================================

def load_dataset(base_path):
    label_map = {'01-minor': 0, '02-moderate': 1, '03-severe': 2}
    
    def load_split(split_name):
        X, y = [], []
        split_path = os.path.join(base_path, split_name)
        print(f"Loading {split_name} data...")
        
        for label_folder in sorted(os.listdir(split_path)):
            if label_folder.startswith('.'):
                continue
            label = label_map[label_folder]
            folder_path = os.path.join(split_path, label_folder)
            
            count = 0
            for img_file in os.listdir(folder_path):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(folder_path, img_file)
                    features = extract_features(img_path)
                    if features is not None:
                        X.append(features)
                        y.append(label)
                        count += 1
            
            print(f"  {label_folder}: {count} images")
        
        return np.array(X), np.array(y)
    
    X_train, y_train = load_split('training')
    print()
    X_test, y_test = load_split('validation')
    
    return X_train, y_train, X_test, y_test

dataset_path = "/Users/stephanieluo/Downloads/data3a"

X_train, y_train, X_test, y_test = load_dataset(dataset_path)

print(f"\n{'='*60}")
print("DATASET LOADED")
print(f"{'='*60}")
print(f"Training: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Test: {X_test.shape[0]} samples")
print(f"Classes: Minor({np.sum(y_train==0)}), Moderate({np.sum(y_train==1)}), Severe({np.sum(y_train==2)})")

# Features
feature_names = []
for color in ['R', 'G', 'B']:
    for i in range(8):
        feature_names.append(f'{color}_hist_bin{i}')
feature_names.extend(['edge_density', 'mean_R', 'std_R', 'mean_G', 'std_G', 
                      'mean_B', 'std_B', 'brightness', 'contrast'])

In [ ]:
# ============================================
# VARIANT 1: GINI
# ============================================

print(f"\n{'='*60}")
print("VARIANT 1: Gini")
print(f"{'='*60}")

dt_shallow = DecisionTreeClassifier(
    max_depth=4,
    criterion='gini',
    random_state=42,
    min_samples_split=20,
    min_samples_leaf=14
)

dt_shallow.fit(X_train, y_train)
y_train_pred_v1 = dt_shallow.predict(X_train)
y_test_pred_v1 = dt_shallow.predict(X_test)

train_acc_v1 = accuracy_score(y_train, y_train_pred_v1)
test_acc_v1 = accuracy_score(y_test, y_test_pred_v1)

print(f"Train Acc: {train_acc_v1:.4f} | Test Acc: {test_acc_v1:.4f}")
print(f"Depth: {dt_shallow.get_depth()} | Leaves: {dt_shallow.get_n_leaves()}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred_v1, 
                          target_names=['Minor', 'Moderate', 'Severe'], digits=3))

cm_v1 = confusion_matrix(y_test, y_test_pred_v1)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_v1, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Minor', 'Moderate', 'Severe'],
            yticklabels=['Minor', 'Moderate', 'Severe'])
plt.title('Confusion Matrix - Variant 1 (Gini)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.savefig('cm_v1.png', dpi=300, bbox_inches='tight')
plt.show()

feature_imp_v1 = pd.DataFrame({
    'feature': feature_names,
    'importance': dt_shallow.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Features:")
print(feature_imp_v1.head(10).to_string(index=False))

plt.figure(figsize=(10, 6))
top_10 = feature_imp_v1.head(10)
plt.barh(range(len(top_10)), top_10['importance'], color='steelblue')
plt.yticks(range(len(top_10)), top_10['feature'])
plt.xlabel('Importance')
plt.title('Feature Importance - Variant 1')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('feat_imp_v1.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================
# VARIANT 2: PRUNED TREE
# ============================================

print(f"\n{'='*60}")
print("VARIANT 2: Pruned Tree")
print(f"{'='*60}")

# Grow deep tree
dt_deep = DecisionTreeClassifier(
    criterion='gini',
    random_state=42,
    min_samples_split=20,
    min_samples_leaf=10
)
dt_deep.fit(X_train, y_train)

print(f"Initial tree: Depth={dt_deep.get_depth()}, Leaves={dt_deep.get_n_leaves()}")

# Pruning path
path = dt_deep.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas

# Find optimal alpha via CV
pruned_trees = []
cv_scores = []
valid_alphas = []

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for alpha in ccp_alphas:
    dt_temp = DecisionTreeClassifier(
        criterion='gini',
        random_state=42,
        ccp_alpha=alpha
    )
    dt_temp.fit(X_train, y_train)
    
    if dt_temp.get_n_leaves() >= 5 and dt_temp.get_n_leaves() < dt_deep.get_n_leaves():
        pruned_trees.append(dt_temp)
        valid_alphas.append(alpha)
        cv_score = cross_val_score(dt_temp, X_train, y_train, cv=kfold).mean()
        cv_scores.append(cv_score)

optimal_idx = np.argmax(cv_scores)
dt_pruned = pruned_trees[optimal_idx]

y_train_pred_v2 = dt_pruned.predict(X_train)
y_test_pred_v2 = dt_pruned.predict(X_test)

train_acc_v2 = accuracy_score(y_train, y_train_pred_v2)
test_acc_v2 = accuracy_score(y_test, y_test_pred_v2)

print(f"\nPruned tree: Depth={dt_pruned.get_depth()}, Leaves={dt_pruned.get_n_leaves()}")
print(f"Train Acc: {train_acc_v2:.4f} | Test Acc: {test_acc_v2:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred_v2, 
                          target_names=['Minor', 'Moderate', 'Severe'], digits=3))

# Confusion Matrix
cm_v2 = confusion_matrix(y_test, y_test_pred_v2)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_v2, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Minor', 'Moderate', 'Severe'],
            yticklabels=['Minor', 'Moderate', 'Severe'])
plt.title('Confusion Matrix - Variant 2 (Pruned)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.savefig('cm_v2.png', dpi=300, bbox_inches='tight')
plt.show()

# Feature
feature_imp_v2 = pd.DataFrame({
    'feature': feature_names,
    'importance': dt_pruned.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Features:")
print(feature_imp_v2.head(10).to_string(index=False))


In [ ]:
# ============================================
# VARIANT 3: ENTROPY
# ============================================

print(f"\n{'='*60}")
print("VARIANT 3: Entropy")
print(f"{'='*60}")

dt_entropy = DecisionTreeClassifier(
    max_depth=4,
    criterion='entropy',
    random_state=42,
    min_samples_split=20,
    min_samples_leaf=14
)

dt_entropy.fit(X_train, y_train)
y_train_pred_v3 = dt_entropy.predict(X_train)
y_test_pred_v3 = dt_entropy.predict(X_test)

train_acc_v3 = accuracy_score(y_train, y_train_pred_v3)
test_acc_v3 = accuracy_score(y_test, y_test_pred_v3)

print(f"Train Acc: {train_acc_v3:.4f} | Test Acc: {test_acc_v3:.4f}")
print(f"Depth: {dt_entropy.get_depth()} | Leaves: {dt_entropy.get_n_leaves()}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred_v3, 
                          target_names=['Minor', 'Moderate', 'Severe'], digits=3))

# Confusion Matrix
cm_v3 = confusion_matrix(y_test, y_test_pred_v3)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_v3, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Minor', 'Moderate', 'Severe'],
            yticklabels=['Minor', 'Moderate', 'Severe'])
plt.title('Confusion Matrix - Variant 3 (Entropy)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.savefig('cm_v3.png', dpi=300, bbox_inches='tight')
plt.show()

# Feature Importance
feature_imp_v3 = pd.DataFrame({
    'feature': feature_names,
    'importance': dt_entropy.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Features:")
print(feature_imp_v3.head(10).to_string(index=False))

In [ ]:
# ============================================
# COMPARISON: ALL THREE VARIANTS
# ============================================

print(f"\n{'='*70}")
print("COMPARISON: ALL THREE VARIANTS")
print(f"{'='*70}\n")

# Comparison Table
comparison_df = pd.DataFrame({
    'Variant': ['Variant 1', 'Variant 2', 'Variant 3'],
    'Description': ['Gini (depth=4)', 'Pruned Tree', 'Entropy (depth=4)'],
    'Criterion': ['Gini', 'Gini (Pruned)', 'Entropy'],
    'Train_Acc': [train_acc_v1, train_acc_v2, train_acc_v3],
    'Test_Acc': [test_acc_v1, test_acc_v2, test_acc_v3],
    'Gen_Gap': [
        abs(train_acc_v1 - test_acc_v1),
        abs(train_acc_v2 - test_acc_v2),
        abs(train_acc_v3 - test_acc_v3)
    ],
    'Depth': [dt_shallow.get_depth(), dt_pruned.get_depth(), dt_entropy.get_depth()],
    'Leaves': [dt_shallow.get_n_leaves(), dt_pruned.get_n_leaves(), dt_entropy.get_n_leaves()]
})

print("MODEL COMPARISON TABLE:")
print(comparison_df.to_string(index=False))
comparison_df.to_csv('model_comparison.csv', index=False)

# Winner
best_idx = comparison_df['Test_Acc'].idxmax()
print(f"\n{'='*70}")
print(f"🏆 WINNER: {comparison_df.loc[best_idx, 'Variant']} - {comparison_df.loc[best_idx, 'Description']}")
print(f"   Test Accuracy: {comparison_df.loc[best_idx, 'Test_Acc']:.4f}")
print(f"{'='*70}\n")

# Figure 1: Train vs Test Accuracy
fig, ax = plt.subplots(figsize=(12, 7))
variants = ['V1\nGini', 'V2\nPruned', 'V3\nEntropy']
train_accs = [train_acc_v1, train_acc_v2, train_acc_v3]
test_accs = [test_acc_v1, test_acc_v2, test_acc_v3]

x = np.arange(len(variants))
width = 0.35

bars1 = ax.bar(x - width/2, train_accs, width, label='Train', color='#4CAF50', alpha=0.8)
bars2 = ax.bar(x + width/2, test_accs, width, label='Test', color='#2196F3', alpha=0.8)

ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Train vs Test Accuracy Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(variants)
ax.legend()
ax.grid(axis='y', alpha=0.3)

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

ax.axvspan(best_idx - 0.4, best_idx + 0.4, alpha=0.15, color='gold')
ax.text(best_idx, 0.95, '★ WINNER', ha='center', fontsize=11, 
        fontweight='bold', color='darkgoldenrod')

plt.tight_layout()
plt.savefig('comparison_accuracy.png', dpi=300, bbox_inches='tight')
plt.show()

# Figure 2: Confusion Matrices
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
cms = [cm_v1, cm_v2, cm_v3]
titles = ['V1: Gini', 'V2: Pruned', 'V3: Entropy']
cmaps = ['Blues', 'Oranges', 'Greens']

for idx, (ax, cm, title, cmap) in enumerate(zip(axes, cms, titles, cmaps)):
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, ax=ax,
                xticklabels=['Minor', 'Mod', 'Severe'],
                yticklabels=['Minor', 'Mod', 'Severe'])
    ax.set_title(f'{title}\nAcc: {[test_acc_v1, test_acc_v2, test_acc_v3][idx]:.3f}', 
                 fontsize=11, fontweight='bold')
    ax.set_ylabel('Actual' if idx == 0 else '')
    ax.set_xlabel('Predicted')
    
    if idx == best_idx:
        for spine in ax.spines.values():
            spine.set_edgecolor('gold')
            spine.set_linewidth(4)

plt.tight_layout()
plt.savefig('comparison_cms.png', dpi=300, bbox_inches='tight')
plt.show()

# Figure 3: Generalization Gap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Test Error
test_errors = [1-test_acc_v1, 1-test_acc_v2, 1-test_acc_v3]
bars = ax1.bar(variants, test_errors, color=['#3498db', '#e74c3c', '#2ecc71'], alpha=0.8)
ax1.set_ylabel('Test Error', fontsize=12, fontweight='bold')
ax1.set_title('Test Error Comparison', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

for bar, err in zip(bars, test_errors):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.005,
             f'{err:.3f}', ha='center', fontsize=10, fontweight='bold')

# Generalization Gap
gen_gaps = comparison_df['Gen_Gap'].values
bars = ax2.bar(variants, gen_gaps, color=['#FF9800', '#FF5722', '#9C27B0'], alpha=0.8)
ax2.set_ylabel('|Train - Test| Accuracy', fontsize=12, fontweight='bold')
ax2.set_title('Generalization Gap', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

for bar, gap in zip(bars, gen_gaps):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.002,
             f'{gap:.4f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('comparison_errors.png', dpi=300, bbox_inches='tight')
plt.show()

# Figure 4: Per-Class F1 Scores Heatmap
metrics_v1 = precision_recall_fscore_support(y_test, y_test_pred_v1, average=None)
metrics_v2 = precision_recall_fscore_support(y_test, y_test_pred_v2, average=None)
metrics_v3 = precision_recall_fscore_support(y_test, y_test_pred_v3, average=None)

f1_matrix = np.array([
    metrics_v1[2],
    metrics_v2[2],
    metrics_v3[2]
])

fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(f1_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

ax.set_xticks(np.arange(3))
ax.set_yticks(np.arange(3))
ax.set_xticklabels(['Minor', 'Moderate', 'Severe'])
ax.set_yticklabels(['Variant 1', 'Variant 2', 'Variant 3'])

for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{f1_matrix[i, j]:.3f}',
               ha='center', va='center', color='black', fontweight='bold')

plt.colorbar(im, ax=ax, label='F1-Score')
ax.set_title('F1-Score by Class and Variant', fontsize=14, fontweight='bold')
ax.set_xlabel('Damage Class', fontsize=12)
ax.set_ylabel('Model Variant', fontsize=12)

plt.tight_layout()
plt.savefig('comparison_f1_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# Figure 5: Model Complexity vs Performance
fig, ax = plt.subplots(figsize=(10, 7))

leaves = comparison_df['Leaves'].values
test_accs_all = comparison_df['Test_Acc'].values

scatter = ax.scatter(leaves, test_accs_all, s=400, 
                    c=['#3498db', '#e74c3c', '#2ecc71'], 
                    alpha=0.6, edgecolors='black', linewidth=2)

for i, txt in enumerate(['V1', 'V2', 'V3']):
    ax.annotate(txt, (leaves[i], test_accs_all[i]),
               ha='center', va='center', fontsize=13, 
               fontweight='bold', color='white')

ax.set_xlabel('Number of Leaves (Model Complexity)', fontsize=12, fontweight='bold')
ax.set_ylabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Model Complexity vs Performance Trade-off', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('comparison_complexity.png', dpi=300, bbox_inches='tight')
plt.show()

# Baseline Comparison
majority_class = Counter(y_train).most_common(1)[0][0]
baseline_acc = accuracy_score(y_test, [majority_class] * len(y_test))

print(f"\nBaseline (majority class): {baseline_acc:.4f}")
print(f"Best model improvement: {(comparison_df.loc[best_idx, 'Test_Acc'] - baseline_acc)*100:.2f}pp")